# DrivenData - Power Laws: Detecting Anomalies in Usage
## Levi Viana

### Introduction :

This notebook takes as input the raw data from the DrivenData Power Laws Challenge sponsored by Schneider Eletric and produces anomaly scores as well as an algorithm to help detecting energy overconsumption.

### Folders :

This notebook relies on the following organization of project folders:

Project -
    - Models (where PyTorch will save the models)
    - Data (where the raw data files should be in)
    - Notebooks (where this notebook should be located)

### Dependencies :

<ol><b>Anaconda3</b>: 
    <li>Download: https://www.anaconda.com/download/#linux</li>
    <li>This code runs in the python 3.5 environment: https://docs.anaconda.com/anaconda/faq#how-do-i-get-the-latest-anaconda-with-python-3-5</li>
</ol>
<ol><b>PyTorch</b>: 
    <li>Download: http://pytorch.org/</li>
    <li>This notebook uses the version 0.3.0.post4 of Pytorch, if is shows errors for newer releases, you should be able to install previous versions of pytorch following these steps http://pytorch.org/previous-versions/</li>
    <li>Be sure that you install the Python 3.5 environment version</li>
</ol>

### Hardware requirement :

<li>CPU RAM: This Notebooks needs about 5 Gb of CPU RAM to run, it can be optimized to run with less memory, but it wasn't the priority of the competition</li>
<li>GPU RAM: It needs about 3,4 Gb of GPU memory to run. The code however should work even if you don't have any GPU installed</li>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix
from random import sample, seed, shuffle
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy
import pickle
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix
from sklearn.decomposition import PCA, KernelPCA, FastICA
import seaborn as sns
import datetime
from scipy import stats

In [2]:
#Checking the PyTorch version installed
print(torch.__version__)

0.3.0.post4


## 1. From raw data to a tidy dataset

In [3]:
# Loading the raw data
weather = pd.read_csv("../Data/weather.csv")
holidays = pd.read_csv("../Data/holidays.csv")
train = pd.read_csv("../Data/train.csv")
metadata = pd.read_csv("../Data/metadata.csv")

FileNotFoundError: File b'../Data/weather.csv' does not exist

In [ ]:
train = train[train['meter_id'] == '234_203']
train['Timestamp'] = pd.to_datetime(train['Timestamp'])
train['Weekday'] = train['Timestamp'].dt.weekday
train['Hour'] = train['Timestamp'].dt.hour
train['Month'] = train['Timestamp'].dt.month

In [ ]:
# Some datasets have duplicates, this cell it take care of it
train = train.groupby(['Timestamp']).first()
train.reset_index(level=0, inplace=True)

In [ ]:
plt.plot(train['Values'].iloc[:1000])
plt.show()

#### 1.1 Adding the Holidays column

In [ ]:
# No holidays reported for this site
holidays['site_id'].unique()

#### 1.2 Adding the Temperature column

In [ ]:
weather['Timestamp'] = pd.to_datetime(weather['Timestamp'])
weather['Date'] = weather['Timestamp'].apply(lambda dt: datetime.datetime(dt.year, 
                                                                      dt.month, 
                                                                      dt.day, 
                                                                      dt.hour))
weather = pd.DataFrame(
    weather[weather['site_id'] == "234_203"].groupby(['Date'])['Temperature'].mean())
weather.reset_index(level=0, inplace=True)
train['Date'] = train['Timestamp'].apply(lambda dt: datetime.datetime(dt.year, 
                                                                      dt.month, 
                                                                      dt.day, 
                                                                      dt.hour))
train = train.merge(weather, how='left', on='Date')
del train['Date']

#### 1.3 Determining buckets of energy consumption (for the predictive model below)

In [ ]:
def consumption_label(x):
    if x >= np.percentile(train['Values'], 99):
        return 'VeryHigh'
    elif x >= np.percentile(train['Values'], 95):
        return 'High'
    elif x >= np.percentile(train['Values'], 80):
        return 'MediumHigh'
    elif x >= np.percentile(train['Values'], 60):
        return 'Medium'
    elif x >= np.percentile(train['Values'], 40):
        return 'MediumLow'
    elif x >= np.percentile(train['Values'], 20):
        return 'Low'
    elif x >= np.percentile(train['Values'], 5):
        return 'VeryLow'
    elif x >= np.percentile(train['Values'], 1):
        return 'StandBy'
    else:
        return 'Off'

In [ ]:
train['Values'] = train['Values'].fillna(-1)
train['target'] = train['Values'].apply(lambda row: consumption_label(row), 1)

#### 1.4 Handling temperature missing values

In [ ]:
 def impute_values(indexes, x):
    """
    This function will be used to fill na's in the temperature column. It will firstly find add "blocks" 
    of na's, i.e, sequence of consecutives na's. Then, it interpolates the missing values by using the 
    temperature values before and after the beggining of the na block.
    
    Arguments:
        indexes: Pandas DataFrame of na's indexes
        x: the Pandas DataFrame column with temperatures
    """
    
    tmp = []
    
    blocks = []
    
    start_block = indexes[0]
    
    for idx in indexes:
        
        if start_block < 0:
            start_block = idx
        
        if (idx+1) in indexes:
            continue
        else:
            blocks.append([start_block, idx])
            start_block = -1
    
    tmp = np.array([])
    for begin, end in blocks:
        
        if begin == 0:
            last_val = x[end+1]
            next_val = x[end+1]
        elif end == len(x)-1:
            last_val = x[begin-1]
            next_val = x[begin-1]
        else:
            last_val = x[begin-1]
            next_val = x[end+1]

        imputation = np.repeat((last_val+next_val)/2, end-begin +1)
        
        tmp = np.concatenate((tmp, imputation))
    
    return tmp

In [ ]:
imputed_temperatures = impute_values(train[train['Temperature'].isnull()].index, 
                                     train['Temperature'])
train.loc[train['Temperature'].isnull(), 'Temperature'] = imputed_temperatures

#### 1.5 Pickling the dataset to gain some time for the next time

In [ ]:
pickle.dump(train, open("../Data/train_prediction_234_203.pkl", "wb"))

In [ ]:
train = pickle.load(open("../Data/train_prediction_234_203.pkl", "rb"))

## 2. Building the predictive model

#### 2.1 Preliminar steps

In [ ]:
features = ['Values', 'Temperature', 'Weekday', 'Hour', 'Month']
feature_scaler = StandardScaler()
feature_scaler.fit(train[features])

In [ ]:
def generate_df(df, sequence_length):
    """
    This function builds a tidy dataset to be used as input to a LSTM. 
    It returns a tensor of the shape [nb_records, 3*sequence_length+2, feature_size] and a vector
    of shape [nb_records] having the target values to predict.
    
    The main idea is to give the model an intraday pattern twice (two previous weeks) and an
    same but incomplete pattern for the current week so that the model has to predict only the
    last value of the pattern.
    
    Arguments:
        df: Pandas DataFrame containing the energy consumption data
        sequence_length: length of the pattern to recognize
    """
    
    target_labels = ['Off', 'StandBy', 'VeryLow', 'Low', 'MediumLow', 'Medium', 'MediumHigh', 'High','VeryHigh']
    target_labels = np.array(target_labels)
    
    X = []
    y = []
    
    for i in range(48*7*2, len(df) - sequence_length - 1):
        for j in range(3):
            if j == 0:
                tmp = feature_scaler.transform(df[features].iloc[i:(i+sequence_length)])
            else:
                ts = feature_scaler.transform(df[features].iloc[(i-j*48*7):(i+sequence_length-j*48*7+1)])
#                 print(ts.shape)
                tmp = np.concatenate((tmp, ts), axis=0)
            
        X.append(tmp)
        y.append(np.where(target_labels == df['target'].iloc[i+sequence_length + 1])[0][0])
        
    return np.array(X), np.array(y)

In [ ]:
train_sequences, train_labels = generate_df(train, 8)
train_sequences.shape

#### 2.2 Splitting train and test datasets

In [ ]:
idx_train, idx_val, _, _ = train_test_split(list(range(train_sequences.shape[0])), 
                                         np.zeros(train_sequences.shape[0]), 
                                         test_size=0.5, 
                                         random_state=2018)

In [ ]:
rnn_sequences = train_sequences[idx_train,:,:]
rnn_labels = train_labels[idx_train]

val_sequences = train_sequences[idx_val,:,:] 
val_labels = train_labels[idx_val]

#### 2.3 Defining the predictive model

In [ ]:
def runValidation(model_dict, cm_print = False):
    model = WinticsRNN(hidden_size=hidden_size, 
                       features_size=len(features))
    
    model.load_state_dict(model_dict)    
    model.eval()
    
    sequences = torch.FloatTensor(val_sequences)
    sequences = autograd.Variable(sequences)
    
    labels_true = torch.LongTensor(val_labels)
    labels_true = autograd.Variable(labels_true)
    
    if torch.cuda.is_available():
        model.cuda()
        sequences = sequences.cuda()
        labels_true = labels_true.cuda()
    
    labels_predicted = model(sequences)
    loss = loss_function(labels_predicted, labels_true)
    
    _, labels_predicted = labels_predicted.max(1)
    labels_predicted = labels_predicted.data.cpu().numpy()
    
    if cm_print:
        print("Current Validation loss: %.3f" % loss.data.cpu().numpy()[0])
        print(confusion_matrix(labels_predicted, val_labels))
    
    return accuracy_score(labels_predicted, val_labels)

In [ ]:
class WinticsRNN(nn.Module):
    def __init__(self, hidden_size, features_size):
        super(WinticsRNN, self).__init__()
        self.hidden_size = hidden_size
        self.features_size = features_size
        
        self.rnn = nn.LSTM(self.features_size, 
                          self.hidden_size, 
                          1, # This argument stands for the nb of layers 
                          dropout=0, 
                          batch_first = True)
        
        self.fc = nn.Linear(self.hidden_size, 9)

    def forward(self, x):
        
        h0 = autograd.Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c0 = autograd.Variable(torch.zeros(1, x.size(0), self.hidden_size))

        if torch.cuda.is_available():
            h0, c0 = h0.cuda(), c0.cuda()

        _, (x, _) = self.rnn(x, (h0, c0))
        
        x = torch.squeeze(x)
        x = self.fc(x)
        
        return x

#### 2.4 Training the model

In [ ]:
nbEpochs = 282
hidden_size = 80
batch_size = 256

net = WinticsRNN(hidden_size=hidden_size, 
                 features_size=len(features))

#### Uncomment the following line if you want to retrain a previously saved model
# net.load_state_dict(torch.load("../Models/LSTM_80_234_203"))

if torch.cuda.is_available():
    net.cuda()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.01)
epoch_losses = []

best_model = net.state_dict()
epoch_acc_list = [] 

for ep in range(nbEpochs):

    avg_loss = []        
            
    net.train()

    for i in range(rnn_sequences.shape[0] // batch_size):
        
        X_batch = rnn_sequences[i*batch_size:(i+1)*batch_size, :, :]
        y_batch = rnn_labels[i*batch_size:(i+1)*batch_size]
        
        # [batch_size, sequence_length, features_size]
        sequences = torch.FloatTensor(X_batch)
        sequences = autograd.Variable(sequences)

        labels_true = torch.LongTensor(y_batch)
        labels_true = autograd.Variable(labels_true)
        
        if torch.cuda.is_available():
            sequences = sequences.cuda()
            labels_true = labels_true.cuda()
        
        labels_predicted = net(sequences)        
        
        net.zero_grad()
        loss = loss_function(labels_predicted, labels_true)
        loss.backward()    
        optimizer.step()

        avg_loss.append(loss.data.cpu().numpy()[0]) 
    
    #<Early stopping>
    epoch_acc = runValidation(net.state_dict())
    epoch_acc_list.append(epoch_acc)

    if max(epoch_acc_list) <= epoch_acc:
        best_model = net.state_dict()

    if max(epoch_acc_list)/1.2 > epoch_acc:
        print("Training stopped by early stopping !")
        print("Validation MSE: %.2f" % (max(epoch_acc_list)))
        plt.plot(epoch_acc_list)
        plt.show()
        break
    #</Early stopping>
    
    if ep % 10 == 1:     
        print("Epoch losses: %.2f" % (np.mean(epoch_losses)))
        if ep % 40 == 1:
            print("Epoch nb: {}".format(ep))
            
            print("Validation Accuracy: %.2f" % runValidation(net.state_dict(), True))
            
            for param_group in optimizer.param_groups:
                param_group['lr'] = param_group['lr'] * 0.8
  
        epoch_losses = []
            
    epoch_losses.append(np.mean(avg_loss))

In [ ]:
torch.save(best_model, "../Models/LSTM_80_234_203")
best_model = torch.load("../Models/LSTM_80_234_203")

#### 2.5 Evaluating the entropy of predictions

In [ ]:
def entropy(model_dict, sequences, labels):
    """
    This function returns (i) the labels predicted by the model as well as (ii) the entropy 
    of predictions, which will serve as an indicator of the prediction's confidence.
    Small entropies mean that the model is sure of what it is predicting, regardless
    the target value.
    
    Arguments:
        model_dict: PyTorch model dictionary, i.e., the output of net.state_dict()
        sequences: RNN inputs, i.e., tensors of the form [nb_records, 3*sequence_length+2, feature_size]
        labels: Target values (only used for some metric evaluation)
    """
    model = WinticsRNN(hidden_size=hidden_size, 
                       features_size=len(features))
    
    model.load_state_dict(model_dict)
    model.eval()
    
    sequences = torch.FloatTensor(sequences)
    sequences = autograd.Variable(sequences)
    
    labels_true = torch.LongTensor(labels)
    labels_true = autograd.Variable(labels_true)

    if torch.cuda.is_available():
        model.cuda()
        sequences = sequences.cuda()
        labels_true = labels_true.cuda()

    labels_predicted = model(sequences)
    
    entropy = [exp_logits/torch.sum(exp_logits) for exp_logits in torch.exp(labels_predicted)]
    entropy = [-torch.log(probs).dot(probs) for probs in entropy]
    entropy = np.array([e.data.cpu().numpy() for e in entropy])
    
    _, labels_predicted = labels_predicted.max(1)
    labels_predicted = labels_predicted.data.cpu().numpy()
    
    print("Dataset accuracy: %.3f" % accuracy_score(labels_predicted, labels))
    cm = confusion_matrix(labels_predicted, labels)
    extended_diagonal = cm.trace()+cm[1:, :-1].trace()+cm[:-1, 1:].trace()
    extended_diagonal_2 = extended_diagonal+cm[2:, :-2].trace()+cm[:-2, 2:].trace()
    
    print("Dataset extended accuracy (max gap 1): %.3f" % (extended_diagonal/np.sum(cm)))
    print("Dataset extended accuracy (max gap 2): %.3f" % (extended_diagonal_2/np.sum(cm)))
    
    return labels_predicted, entropy

In [ ]:
# Calculating the entropies for the training dataset
overconsumption_train, entropy_train = entropy(best_model, rnn_sequences, rnn_labels)

In [ ]:
# Calculating the entropies for the test dataset
overconsumption_val, entropy_val = entropy(best_model, val_sequences, val_labels)

In [ ]:
# Creating a column 'entropy' in the original dataset 
train['entropy'] = 3.0
train.loc[48*7*2+8+1+np.array(idx_val), 'entropy'] = entropy_val[:, 0]
train.loc[48*7*2+8+1+np.array(idx_train), 'entropy'] = entropy_train[:, 0]

In [ ]:
# Creating a column 'predicted' in the original dataset
target_labels = ['Off', 'StandBy', 'VeryLow', 'Low', 'MediumLow', 'Medium', 'MediumHigh', 'High','VeryHigh']
train['predicted'] = "None"
train.loc[48*7*2+8+1+np.array(idx_val), 'predicted'] = [target_labels[v] for v in overconsumption_val]
train.loc[48*7*2+8+1+np.array(idx_train), 'predicted'] = [target_labels[v] for v in overconsumption_train]

#### 2.6 Determining the anomaly score

In [ ]:
def get_gap(row):
    """
    This function returns the number of buckets lying between the prediction and the target value.
    It will be applied to all the main dataset's lines.
    
    Arguments:
        row: row of the dataset.
    """
    if row['predicted'] in target_labels:
        idx_pred = np.where(np.array(target_labels) == row['predicted'])[0][0]
        idx_true = np.where(np.array(target_labels) == row['target'])[0][0]

        return idx_true - idx_pred
    else:
        return 0

In [ ]:
train['gap'] = train.apply(lambda row: get_gap(row), 1)

##### CRITERIA : the anomaly score

The main motivation for that anomaly score is to catch two behaviours :

    - Low entropy and low prediction gap:
        The model is sure of what it is predicting, and still, there is a gap. This effect
        increases the risk that the current measurement is actually an anomaly.
    - Moderate enropy and high prediction gap:
        If the gap is somehow higher than what the entropy measurement is expecting,
        then the this energy measurement is a potential anomaly.

Since there are 9 buckets, the maximum entropy we can possibly get is ln(9), which is about 2.2.

Let's consider an arbitrary prediction. Suppose that its entropy is equal to 0.69, then, 
the mass of probabilities is very likely distributed to only 2 buckets (because ln(2) is about 0.69). 
Suppose that the gap between the prediction and the target value is 3. We have then a suspicious
energy measurement: the probabilities are very high for only two buckets (probably one next 
to the other), and still the target value is 3 buckets away from the prediction.

The same logic can be drawn for all gaps. I came up with the following formula:
$$gap*exp(ln(9)-entropy)$$

It will have high absolute values whenever something unexpected happens.

In [ ]:
train['criteria'] = train['gap']*np.exp(np.log(9) - train['entropy'])

#### 2.7 Rule-based approach (Percentiles)

In [ ]:
train['temp_bucket'] = train['Temperature'].apply(lambda row: np.round(row/5)*5)
train['working_day'] = train['Weekday'] < 5

In [ ]:
def get_percentile(row):
    """
    This function returns the percentile of the energy consumption measurement with respect
    to (i) the Temperature, (ii) the type of day and (iii) the hour.
    
    Arguments:
        row: row of the dataset.
    """
    bucket = row['temp_bucket']
    weekday = row['working_day']
    hour = row['Hour']
    
    cond = (train['temp_bucket'] == bucket)&(train['working_day'] == weekday)&(train['Hour'] == hour)    
    range_of_values = train.loc[cond, 'Values']
    
    return stats.percentileofscore(range_of_values, row['Values'])

In [ ]:
train['percentile'] = train.apply(lambda row: get_percentile(row), 1)

#### 2.8 Rule-based approach (Checking the context)

In [ ]:
 def check_context(idx):
    """
    The main idea of that function is to detect and eliminate anomalies that come from abnormal
    contexts. For instance, if there is a conference takes place in the building for an entire
    week, the energy consumption levels will be higher than the 'normal' days but it won't
    necessarily be classified as overconsumption. So the idea is to detect these kind of weeks 
    and ignore any alarm that the predictive model might throw in that period of time.
    
    This function checks whether the pattern of a certain meter neighbourhood value is high 
    across the entire week. It compares the pattern percentiles for the precedent week against
    the two former weeks. It returns the number of days it went higher.
    
    Arguments:
        idx: index of the meter measurement, i.e., dataset row.
    """
    ans = 0
    
    for j in range(7):
        week_n   = train.loc[idx-8-j*96       :idx+8-j*96, 'percentile'].mean()
        week_n_1 = train.loc[idx-8-j*96-96*7*1:idx+8-j*96-96*7*1, 'percentile'].mean()
        week_n_2 = train.loc[idx-8-j*96-96*7*2:idx+8-j*96-96*7*2, 'percentile'].mean()
        
        ans += (week_n > week_n_1 + 10)*1 + (week_n > week_n_2 + 10)*1
    
    return ans

In [ ]:
# Here is an illustration of what can be detected by this model

fig, ax1 = plt.subplots(figsize=(16, 8))
begin = 28896-96
end = 28896+1+96
consumption = train.loc[begin:end, 'Values'].values - train.loc[begin:end, 'Values'].min()
consumption = 100*consumption/(train.loc[begin:end, 'Values'].max()-train.loc[begin:end, 'Values'].min())
slide = 28896
ax1.plot(train.loc[begin:end, 'percentile'].values)
ax1.plot(consumption, color = "green")
ax1.set_ylim([0, 100])
ax2 = ax1.twinx()
ax2.plot(train.loc[begin:end, 'criteria'].values, color="red", linestyle=":", linewidth=3)
ax2.set_ylim([-16, 16])
plt.show()

#### 2.9 The algorithm

In [ ]:
# Initialization
train['is_abnormal'] = False

# Detecting the beggining of anomalies (Prediction-Based approach)
train.loc[train['criteria']>11, 'is_abnormal'] = True

# Filtering process (Rule-based approach)
for idx in train[train['is_abnormal']].index:
    
    # Eliminating abnormal energy consumption contexts
    if check_context(idx) > 8:
        train.loc[idx, 'is_abnormal'] = False
        continue
    
    # Eliminating low percentile anomalies
    if train.loc[idx, 'percentile'] < 90:
        train.loc[idx, 'is_abnormal'] = False
        continue
    
    # Eliminating anomalies that come just after na values
    if train.loc[idx-5:idx-1, 'Values'].min() < 0:
        train.loc[idx, 'is_abnormal'] = False
        continue
    
    # Anomalies should begin just after a ascending percentile pattern
    if train.loc[idx-5:idx-1, 'percentile'].mean() >= 80:
        train.loc[idx, 'is_abnormal'] = False

# Extending the duration of the anomalies
for idx in train[train['is_abnormal']].index:
    k=0
    moving_average = train.loc[idx, 'percentile']
    while (moving_average >= 80):
        train.loc[idx+k, 'is_abnormal'] = True
        k+=1
        moving_average = 0.2*moving_average + 0.8*train.loc[idx+k, 'percentile']

    # Eliminating anomalies that last less than an hour
    if k < 2:
        train.loc[idx:idx+k, 'is_abnormal'] = False

In [ ]:
sum(train['is_abnormal'])

#### 2.10 Illustration of the anomalies found

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 8))
begin = 56325-96
end = 56325+1+96
ax1.plot(train.loc[begin:end, 'percentile'])
ax1.plot(train[train['is_abnormal']].loc[begin:end, 'percentile'], color = "green", linewidth=10, linestyle=":")
ax1.set_ylim([0, 100])
ax2 = ax1.twinx()
ax2.plot(train.loc[begin:end, 'criteria'], color="red")
ax2.set_ylim([-16, 16])
plt.show()